In [242]:
import pandas as pd

In [243]:
url = "../data/data_UFC_EDA/Estadisticas_peleador/estadisticas_peleador.csv"
df= pd.read_csv(url)
df.head()

,name,wins,losses,height,weight,reach,stance,age,SLpM,sig_str_acc,SApM,str_def,td_avg,td_acc,td_def,sub_avg
0,Amanda Ribas,12.0,5.0,160.02,56.70,167.64,Orthodox,30.0,4.63,0.40,3.40,0.61,2.07,0.51,0.85,0.7
1,Rose Namajunas,13.0,6.0,165.10,56.70,165.10,Orthodox,31.0,3.69,0.41,3.51,0.63,1.38,0.47,0.59,0.5
2,Karl Williams,10.0,1.0,190.50,106.59,200.66,Orthodox,34.0,2.87,0.52,1.70,0.60,4.75,0.50,1.00,0.2
3,Justin Tafa,7.0,4.0,182.88,119.75,187.96,Southpaw,30.0,4.09,0.54,5.02,0.47,0.00,0.00,0.50,0.0
4,Edmen Shahbazyan,13.0,4.0,187.96,83.91,190.50,Orthodox,26.0,3.60,0.52,4.09,0.45,2.24,0.38,0.63,0.6


In [244]:
df.columns

Index(['name', 'wins', 'losses', 'height', 'weight', 'reach', 'stance', 'age',
       'SLpM', 'sig_str_acc', 'SApM', 'str_def', 'td_avg', 'td_acc', 'td_def',
       'sub_avg'],
      dtype='object')

In [245]:
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [246]:
df = df.drop(columns = ['wins','losses','reach',"age"])

In [247]:
df['stance'] = df['stance'].fillna("Orthodox")

In [248]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2479 entries, 0 to 2478
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         2478 non-null   object 
 1   height       2478 non-null   float64
 2   weight       2478 non-null   float64
 3   stance       2479 non-null   object 
 4   SLpM         2478 non-null   float64
 5   sig_str_acc  2478 non-null   float64
 6   SApM         2478 non-null   float64
 7   str_def      2478 non-null   float64
 8   td_avg       2478 non-null   float64
 9   td_acc       2478 non-null   float64
 10  td_def       2478 non-null   float64
 11  sub_avg      2478 non-null   float64
dtypes: float64(10), object(2)
memory usage: 232.5+ KB


In [249]:
df["stance"].value_counts()

stance
Orthodox       1918
Southpaw        412
Switch          140
Open Stance       6
Sideways          3
Name: count, dtype: int64

In [250]:
df["stance"] = df["stance"].str.strip()
stance_dict = {"Orthodox":1,
               "Southpaw":2,
               "Switch":3,
               "Open Stance":4,
               "SideWays":5}

df["stance"] = df["stance"].map(stance_dict)

In [251]:
df["stance"].value_counts()

stance
1.0    1918
2.0     412
3.0     140
4.0       6
Name: count, dtype: int64

In [252]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2479 entries, 0 to 2478
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         2478 non-null   object 
 1   height       2478 non-null   float64
 2   weight       2478 non-null   float64
 3   stance       2476 non-null   float64
 4   SLpM         2478 non-null   float64
 5   sig_str_acc  2478 non-null   float64
 6   SApM         2478 non-null   float64
 7   str_def      2478 non-null   float64
 8   td_avg       2478 non-null   float64
 9   td_acc       2478 non-null   float64
 10  td_def       2478 non-null   float64
 11  sub_avg      2478 non-null   float64
dtypes: float64(11), object(1)
memory usage: 232.5+ KB


In [253]:
df["name"]

0           Amanda Ribas
1         Rose Namajunas
2          Karl Williams
3            Justin Tafa
4       Edmen Shahbazyan
              ...       
2474     Thaddeus Luster
2475       David Levicki
2476          Ray Wizard
2477      Sean Daugherty
2478                 NaN
Name: name, Length: 2479, dtype: object

In [254]:
import pandas as pd

def comparar_pelea(df, fighter1, fighter2):

    # Filtramos las filas para cada peleador
    data_fighter1 = df[df['name'] == fighter1].copy()
    data_fighter2 = df[df['name'] == fighter2].copy()

    # Cambiamos los nombres de las columnas para cada peleador
    data_fighter1.columns = [col + '_fighter1' for col in data_fighter1.columns]
    data_fighter2.columns = [col + '_fighter2' for col in data_fighter2.columns]

    # Concatenamos los datos de los peleadores en una sola fila
    comparison_df = pd.concat([data_fighter1.reset_index(drop=True), data_fighter2.reset_index(drop=True)], axis=1)

    # Inicializamos el DataFrame de resultados con los nombres de los peleadores
    comparison_result = pd.DataFrame({
        'red_fighter': [fighter1],
        'blue_fighter': [fighter2]
    })

    # Calcular las diferencias solo en las columnas numéricas
    for col1, col2 in zip(data_fighter1.columns, data_fighter2.columns):
        # Verificar si las columnas son numéricas
        if '_fighter1' in col1 and '_fighter2' in col2:
            # Convertir a valores numéricos, 'coerce' convertirá los errores a NaN
            metric_name = col1.replace('_fighter1', '')
            if pd.to_numeric(comparison_df[col1], errors='coerce').dtype != 'O' and pd.to_numeric(comparison_df[col2], errors='coerce').dtype != 'O':
                comparison_result[metric_name + '_diff'] = pd.to_numeric(comparison_df[col1], errors='coerce') - pd.to_numeric(comparison_df[col2], errors='coerce')

    return comparison_result


In [255]:
lucha =comparar_pelea(df,"Amanda Ribas","Rose Namajunas")
lucha.columns
lucha

,red_fighter,blue_fighter,name_diff,height_diff,weight_diff,stance_diff,SLpM_diff,sig_str_acc_diff,SApM_diff,str_def_diff,td_avg_diff,td_acc_diff,td_def_diff,sub_avg_diff
0,Amanda Ribas,Rose Namajunas,NaN,-5.08,0.0,0.0,0.94,-0.01,-0.11,-0.02,0.69,0.04,0.26,0.2
